In [10]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
df = pd.read_csv('infl_rus.csv')
df = df.iloc[:, 2:]

# Стандартизация данных (PCA требует стандартизации)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Применение PCA
pca = PCA()
pca.fit(df_scaled)

# Преобразование данных в главные компоненты
components = pca.transform(df_scaled)

# Вывод объясненной дисперсии каждой компоненты
explained_variance = pca.explained_variance_ratio_
for i, variance in enumerate(explained_variance):
    print(f'Компонента {i+1}: {variance:.2%} объясненной дисперсии')

# Вывод весов каждого параметра в главных компонентах
weights = pca.components_
for i, component in enumerate(weights):
    print(f'\nКомпонента {i+1}:')
    for j, weight in enumerate(component):
        print(f'{df.columns[j]}: {weight:.4f}')

# Интерпретация влияния факторов на инфляцию
# Обычно первая компонента объясняет наибольшую дисперсию и может быть использована для интерпретации
most_important_component = weights[0]
print('\nНаиболее важная компонента:')
for j, weight in enumerate(most_important_component):
    print(f'{df.columns[j]}: {weight:.4f} (влияние на инфляцию: {weight*explained_variance[0]:.2%})')

Компонента 1: 44.22% объясненной дисперсии
Компонента 2: 24.22% объясненной дисперсии
Компонента 3: 15.40% объясненной дисперсии
Компонента 4: 9.64% объясненной дисперсии
Компонента 5: 5.56% объясненной дисперсии
Компонента 6: 0.81% объясненной дисперсии
Компонента 7: 0.16% объясненной дисперсии

Компонента 1:
Российская Федерация: 0.1991
us_rate: 0.4834
MIACR: 0.3559
Infl_exp: 0.1316
M2: 0.5244
gov_debt: 0.5158
Price: 0.2041

Компонента 2:
Российская Федерация: 0.5169
us_rate: -0.1358
MIACR: 0.4479
Infl_exp: 0.5999
M2: -0.2716
gov_debt: -0.2784
Price: 0.0508

Компонента 3:
Российская Федерация: 0.0544
us_rate: -0.4198
MIACR: -0.1109
Infl_exp: -0.0813
M2: 0.0549
gov_debt: 0.0612
Price: 0.8917

Компонента 4:
Российская Федерация: 0.7726
us_rate: 0.1706
MIACR: -0.1803
Infl_exp: -0.5698
M2: -0.1061
gov_debt: -0.0685
Price: -0.0299

Компонента 5:
Российская Федерация: 0.2009
us_rate: 0.1228
MIACR: -0.7723
Infl_exp: 0.5316
M2: -0.0402
gov_debt: 0.2522
Price: -0.0169

Компонента 6:
Российска

In [12]:
df

,Российская Федерация,us_rate,MIACR,Infl_exp,M2,gov_debt,Price
0,100.39,33.784353,6.056471,13.037736,31155.6,5722.239,108.12
1,100.49,35.244040,5.993500,11.795154,29861.2,5733.566,108.90
2,100.82,36.198565,7.845000,11.522133,30169.0,5754.951,107.48
3,100.87,35.667727,7.874545,12.672745,29519.3,5707.239,107.76
4,100.85,34.833655,8.214737,11.720957,29879.0,5714.967,109.54
...,...,...,...,...,...,...,...
127,100.60,89.160161,17.760909,12.520500,104351.3,21760.809,80.36
128,100.75,91.234505,18.460000,13.412700,106242.7,21537.407,74.02
129,100.81,96.160226,18.925217,13.366700,107629.5,21688.947,75.63
130,101.10,100.687450,20.645714,13.941200,108947.9,23742.293,74.35


In [13]:
df = pd.read_csv('infl_rus.csv')
df = df.iloc[:, 2:]

# Разделяем данные на признаки (X) и целевую переменную (y)
X = df.drop('Российская Федерация', axis=1)
y = df['Российская Федерация']

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем и обучаем модель CatBoostRegressor
model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=3, verbose=0)
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
print(f'Среднеквадратичная ошибка (MSE): {mse:.4f}')

# Важность признаков
feature_importance = model.get_feature_importance()
feature_names = X.columns

# Вывод важности признаков
print('\nВажность признаков:')
for feature, importance in zip(feature_names, feature_importance):
    print(f'{feature}: {importance:.2f}')

# Интерпретация влияния факторов на инфляцию
total_importance = sum(feature_importance)
print('\nВлияние факторов на инфляцию:')
for feature, importance in zip(feature_names, feature_importance):
    print(f'{feature}: {importance / total_importance * 100:.2f}%')

Среднеквадратичная ошибка (MSE): 2.1491

Важность признаков:
us_rate: 17.96
MIACR: 27.49
Infl_exp: 19.22
M2: 17.71
gov_debt: 13.50
Price: 4.13

Влияние факторов на инфляцию:
us_rate: 17.96%
MIACR: 27.49%
Infl_exp: 19.22%
M2: 17.71%
gov_debt: 13.50%
Price: 4.13%
